In [1]:
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pandas import DataFrame
import json
from datetime import datetime
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import matplotlib.ticker as mtick
from mpl_toolkits.axes_grid1 import host_subplot
from mpl_toolkits import axisartist
import mpl_axes_aligner

In [2]:
low_memory=False
file_to_load = "world_mask_mandates.csv"
# Read Purchasing File and store into Pandas data frame
data_raw= pd.read_csv(file_to_load)
data_raw["ConfirmedCases"] = data_raw["ConfirmedCases"].replace(np.nan, 0)
# data_raw.head()

In [3]:
# data_raw.columns

In [4]:
data_reduced= data_raw[['CountryName','RegionName','Jurisdiction','Date','H6_combined_numeric','H6_combined','ConfirmedCases','ConfirmedDeaths']]
data_reduced.head(5)

,CountryName,RegionName,Jurisdiction,Date,H6_combined_numeric,H6_combined,ConfirmedCases,ConfirmedDeaths
0,Aruba,NaN,NAT_TOTAL,20200101,0.0,0,0.0,NaN
1,Aruba,NaN,NAT_TOTAL,20200102,0.0,0,0.0,NaN
2,Aruba,NaN,NAT_TOTAL,20200103,0.0,0,0.0,NaN
3,Aruba,NaN,NAT_TOTAL,20200104,0.0,0,0.0,NaN
4,Aruba,NaN,NAT_TOTAL,20200105,0.0,0,0.0,NaN


In [5]:
# data_reduced.dtypes

In [6]:
data_reduced.insert(5, 'daily_cases','')
data_reduced.head()

,CountryName,RegionName,Jurisdiction,Date,H6_combined_numeric,daily_cases,H6_combined,ConfirmedCases,ConfirmedDeaths
0,Aruba,NaN,NAT_TOTAL,20200101,0.0,,0,0.0,NaN
1,Aruba,NaN,NAT_TOTAL,20200102,0.0,,0,0.0,NaN
2,Aruba,NaN,NAT_TOTAL,20200103,0.0,,0,0.0,NaN
3,Aruba,NaN,NAT_TOTAL,20200104,0.0,,0,0.0,NaN
4,Aruba,NaN,NAT_TOTAL,20200105,0.0,,0,0.0,NaN


In [7]:
# Formating Date column.
data_reduced['Date'] = pd.to_datetime(data_reduced['Date'],format ='%Y%m%d')

In [8]:
# truncking the date range
A = '2020-01-01'
B = '2021-12-31'
data_truncated = data_reduced.loc[(data_reduced['Date'] >= A) & (data_reduced['Date'] <= B)]
# data_truncated.head(5)

In [9]:
data_six = data_truncated.loc[data_truncated['CountryName'].isin(['United States']) & data_truncated['Jurisdiction'].isin(['STATE_TOTAL']) ]
data_six['CountryName']

192528    United States
192529    United States
192530    United States
192531    United States
192532    United States
              ...      
231454    United States
231455    United States
231456    United States
231457    United States
231458    United States
Name: CountryName, Length: 37281, dtype: object

In [10]:
data_six

,CountryName,RegionName,Jurisdiction,Date,H6_combined_numeric,daily_cases,H6_combined,ConfirmedCases,ConfirmedDeaths
192528,United States,Alaska,STATE_TOTAL,2020-01-01,0.0,,0,0.0,NaN
192529,United States,Alaska,STATE_TOTAL,2020-01-02,0.0,,0,0.0,NaN
192530,United States,Alaska,STATE_TOTAL,2020-01-03,0.0,,0,0.0,NaN
192531,United States,Alaska,STATE_TOTAL,2020-01-04,0.0,,0,0.0,NaN
192532,United States,Alaska,STATE_TOTAL,2020-01-05,0.0,,0,0.0,NaN
...,...,...,...,...,...,...,...,...,...
231454,United States,Wyoming,STATE_TOTAL,2021-12-27,1.5,,2T,114624.0,1526.0
231455,United States,Wyoming,STATE_TOTAL,2021-12-28,1.5,,2T,114917.0,1526.0
231456,United States,Wyoming,STATE_TOTAL,2021-12-29,1.5,,2T,115242.0,1526.0
231457,United States,Wyoming,STATE_TOTAL,2021-12-30,1.5,,2T,115638.0,1526.0


In [18]:
states_totals = data_six.groupby(['RegionName']).max()["ConfirmedCases"]
states_totals

RegionName
Alabama            896614.0
Alaska             157169.0
Arizona           1381488.0
Arkansas           566486.0
California        5522233.0
Colorado           929275.0
Connecticut        510188.0
Delaware           180366.0
Florida           4209927.0
Georgia           1839879.0
Hawaii             112932.0
Idaho              319382.0
Illinois          2149548.0
Indiana           1246854.0
Iowa               575501.0
Kansas             520388.0
Kentucky           856145.0
Louisiana          828695.0
Maine              146736.0
Maryland           700553.0
Massachusetts     1140614.0
Michigan          1710325.0
Minnesota         1022212.0
Mississippi        543737.0
Missouri          1013458.0
Montana            197724.0
Nebraska           338257.0
Nevada             484641.0
New Hampshire      198667.0
New Jersey        1564253.0
New Mexico         350043.0
New York          3480280.0
North Carolina    1686667.0
North Dakota       174175.0
Ohio              2016082.0
Oklahoma 

# United States

In [12]:
# usa_df = data_six.loc[data_six['CountryName'].isin(['United States'])]
# usa_df['daily_cases'] =usa_df['ConfirmedCases'].diff()
# usa_df['daily_cases'] =usa_df['daily_cases'].fillna(0)

# x1_usa = usa_df[['H6_combined_numeric']]
# x2_usa = usa_df[['daily_cases']]
# dates_usa = usa_df[['Date']]

# fig, ax = plt.subplots(figsize=(100, 30))
# ax.plot(dates_usa,x1_usa,linewidth=7,color="r",label='Mask Mandates')
# ax.tick_params('y',colors='r',labelsize=40)
# ax.tick_params('x',color='k',labelsize=40)
# ax.set_title('UNITED STATES',fontsize = 100)
# ax.set_xlabel('Dates', fontsize=60)
# ax.xaxis.set_label_coords(.5, -.06)
# ax.set_ylabel('Mandate Leves', fontsize=60)
# ax.yaxis.set_label_coords(-0.02, .5)
# ax.set_facecolor('xkcd:white')
# ax.set_ylim(top = 4.1)

# ax2 =ax.twinx()
# ax2.plot(dates_usa,x2_usa,linewidth=4,color="b",label='Daily Confirmed Cases')
# ax2.set_ylabel('Confirmed Cases', fontsize=60)
# ax2.yaxis.set_label_coords(1.04, .5)
# ax2.tick_params('y', colors='b',labelsize=40)
# ax2.yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))

# mpl_axes_aligner.align.yaxes(ax, 0, ax2, 0, 0.05)

# lines, labels = ax.get_legend_handles_labels()
# lines2, labels2 = ax2.get_legend_handles_labels()
# ax2.legend(lines + lines2, labels + labels2, loc='best',fontsize=60)
# ax2.grid(True)
# plt.savefig("United States.png")

# CANADA

In [13]:
# can_df = data_six.loc[data_six['CountryName'].isin(['Canada'])]
# can_df['daily_cases'] =can_df['ConfirmedCases'].diff()
# can_df['daily_cases'] =can_df['daily_cases'].fillna(0)

# x1_can = can_df[['H6_combined_numeric']]
# x2_can = can_df[['daily_cases']]
# dates_can = can_df[['Date']]

# fig, ax = plt.subplots(figsize=(100, 30))
# ax.plot(dates_can,x1_can,linewidth=7,color="r",label='Mask Mandates')
# ax.tick_params('y',colors='r',labelsize=40)
# ax.tick_params('x',color='k',labelsize=40)
# ax.set_title('CANADA',fontsize = 100)
# ax.set_xlabel('Dates', fontsize=60)
# ax.xaxis.set_label_coords(.5, -.06)
# ax.set_ylabel('Mandate Leves', fontsize=60)
# ax.yaxis.set_label_coords(-0.02, .5)
# ax.set_facecolor('xkcd:white')
# ax.set_ylim(top = 4.1)
# ax2 =ax.twinx()
# ax2.plot(dates_can,x2_can,linewidth=4,color="b",label='Daily Confirmed Cases')
# ax2.set_ylabel('Confirmed Cases', fontsize=60)
# ax2.yaxis.set_label_coords(1.04, .5)
# ax2.tick_params('y', colors='b',labelsize=40)
# ax2.yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))

# mpl_axes_aligner.align.yaxes(ax, 0, ax2, 0, 0.05)

# lines, labels = ax.get_legend_handles_labels()
# lines2, labels2 = ax2.get_legend_handles_labels()
# ax2.legend(lines + lines2, labels + labels2, loc='best',fontsize=60)
# ax2.grid(True)
# plt.savefig("Canada.png")

# GERMANY

In [14]:
# ger_df = data_six.loc[data_six['CountryName'].isin(['Germany'])]
# ger_df['daily_cases'] =ger_df['ConfirmedCases'].diff()
# ger_df['daily_cases'] =ger_df['daily_cases'].fillna(0)

# x1_ger = ger_df[['H6_combined_numeric']]
# x2_ger = ger_df[['daily_cases']]
# dates_ger = ger_df[['Date']]

# fig, ax = plt.subplots(figsize=(100, 30))
# ax.plot(dates_ger,x1_ger,linewidth=7,color="r",label='Mask Mandates')
# ax.tick_params('y',colors='r',labelsize=40)
# ax.tick_params('x',color='k',labelsize=40)
# ax.set_title('GERMANY',fontsize = 100)
# ax.set_xlabel('Dates', fontsize=60)
# ax.xaxis.set_label_coords(.5, -.06)
# ax.set_ylabel('Mandate Leves', fontsize=60)
# ax.yaxis.set_label_coords(-0.02, .5)
# ax.set_facecolor('xkcd:white')
# ax.set_ylim(top = 4.1)
# ax2 =ax.twinx()
# ax2.plot(dates_ger,x2_ger,linewidth=4,color="b",label='Daily Confirmed Cases')
# ax2.set_ylabel('Confirmed Cases', fontsize=60)
# ax2.yaxis.set_label_coords(1.04, .5)
# ax2.tick_params('y', colors='b',labelsize=40)
# ax2.yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))

# mpl_axes_aligner.align.yaxes(ax, 0, ax2, 0, 0.05)

# lines, labels = ax.get_legend_handles_labels()
# lines2, labels2 = ax2.get_legend_handles_labels()
# ax2.legend(lines + lines2, labels + labels2, loc='best',fontsize=60)
# ax2.grid(True)
# plt.savefig("Germany.png")

# JAPAN

In [15]:
# jap_df = data_six.loc[data_six['CountryName'].isin(['Japan'])]
# jap_df['daily_cases'] =jap_df['ConfirmedCases'].diff()
# jap_df['daily_cases'] =jap_df['daily_cases'].fillna(0)

# x1_jap = jap_df[['H6_combined_numeric']]
# x2_jap = jap_df[['daily_cases']]
# dates_jap = jap_df[['Date']]

# fig, ax = plt.subplots(figsize=(100, 30))
# ax.plot(dates_jap,x1_jap,linewidth=7,color="r",label='Mask Mandates')
# ax.tick_params('y',colors='r',labelsize=40)
# ax.tick_params('x',color='k',labelsize=40)
# ax.set_title('JAPAN',fontsize = 100)
# ax.set_xlabel('Dates', fontsize=60)
# ax.xaxis.set_label_coords(.5, -.06)
# ax.set_ylabel('Mandate Leves', fontsize=60)
# ax.yaxis.set_label_coords(-0.02, .5)
# ax.set_facecolor('xkcd:white')
# ax.set_ylim(top = 4.1)

# ax2 =ax.twinx()
# ax2.plot(dates_jap,x2_jap,linewidth=4,color="b",label='Daily Confirmed Cases')
# ax2.set_ylabel('Confirmed Cases', fontsize=60)
# ax2.yaxis.set_label_coords(1.04, .5)
# ax2.tick_params('y', colors='b',labelsize=40)
# ax2.yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))

# mpl_axes_aligner.align.yaxes(ax, 0, ax2, 0, 0.05)

# lines, labels = ax.get_legend_handles_labels()
# lines2, labels2 = ax2.get_legend_handles_labels()
# ax2.legend(lines + lines2, labels + labels2, loc='upper left',fontsize=60)
# ax2.grid(True)
# plt.savefig("Japan.png")

# INDIA

In [16]:
# ind_df = data_six.loc[data_six['CountryName'].isin(['India'])]
# ind_df['daily_cases'] =ind_df['ConfirmedCases'].diff()
# ind_df['daily_cases'] =ind_df['daily_cases'].fillna(0)

# x1_ind = ind_df[['H6_combined_numeric']]
# x2_ind = ind_df[['daily_cases']]
# dates_ind = ind_df[['Date']]

# fig, ax = plt.subplots(figsize=(100, 30))
# ax.plot(dates_ind,x1_ind,linewidth=7,color="r",label='Mask Mandates')
# ax.tick_params('y',colors='r',labelsize=40)
# ax.tick_params('x',color='k',labelsize=40)
# ax.set_title('INDIA',fontsize = 100)
# ax.set_xlabel('Dates', fontsize=60)
# ax.xaxis.set_label_coords(.5, -.06)
# ax.set_ylabel('Mandate Leves', fontsize=60)
# ax.yaxis.set_label_coords(-0.02, .5)
# ax.set_facecolor('xkcd:white')
# ax.set_ylim(top = 4.1)

# ax2 =ax.twinx()
# ax2.plot(dates_ind,x2_ind,linewidth=4,color="b",label='Daily Confirmed Cases')
# ax2.set_ylabel('Confirmed Cases', fontsize=60)
# ax2.yaxis.set_label_coords(1.04, .5)
# ax2.tick_params('y', colors='b',labelsize=40)
# ax2.yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))

# mpl_axes_aligner.align.yaxes(ax, 0, ax2, 0, 0.05)

# lines, labels = ax.get_legend_handles_labels()
# lines2, labels2 = ax2.get_legend_handles_labels()
# ax2.legend(lines + lines2, labels + labels2, loc='upper left',fontsize=60)
# ax2.grid(True)
# plt.savefig("India.png")

# SWEDEN

In [17]:
# swe_df = data_six.loc[data_six['CountryName'].isin(['Sweden'])]
# swe_df['daily_cases'] =swe_df['ConfirmedCases'].diff()
# swe_df['daily_cases'] =swe_df['daily_cases'].fillna(0)

# x1_swe = swe_df[['H6_combined_numeric']]
# x2_swe = swe_df[['daily_cases']]
# dates_swe = swe_df[['Date']]

# fig, ax = plt.subplots(figsize=(100, 30))
# ax.plot(dates_swe,x1_swe,linewidth=7,color="r",label='Mask Mandates')
# ax.tick_params('y',colors='r',labelsize=40)
# ax.tick_params('x',color='k',labelsize=40)
# ax.set_title('SWEDEN',fontsize = 100)
# ax.set_xlabel('Dates', fontsize=60)
# ax.xaxis.set_label_coords(.5, -.06)
# ax.set_ylabel('Mandate Leves', fontsize=60)
# ax.yaxis.set_label_coords(-0.02, .5)
# ax.set_facecolor('xkcd:white')
# ax.set_ylim(top = 4.1)

# ax2 =ax.twinx()
# ax2.plot(dates_swe,x2_swe,linewidth=4,color="b",label='Daily Confirmed Cases')
# ax2.set_ylabel('Confirmed Cases', fontsize=60)
# ax2.yaxis.set_label_coords(1.04, .5)
# ax2.tick_params('y', colors='b',labelsize=40)
# ax2.yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))

# mpl_axes_aligner.align.yaxes(ax, 0, ax2, 0, 0.05)

# lines, labels = ax.get_legend_handles_labels()
# lines2, labels2 = ax2.get_legend_handles_labels()
# ax2.legend(lines + lines2, labels + labels2, loc='upper left',fontsize=60)
# ax2.grid(True)
# plt.savefig("Sweden.png")